# PLN Review App - Sentiment Analysis

Nama: Indri Windriasari <br>
Email: indriwindriasari2511@gmail.com

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
csv_file = 'pln-mobile3.csv'
df_pln = pd.read_csv(csv_file)

In [ ]:
df_pln.sample(2)

,Unnamed: 0.1,Unnamed: 0,content,score,at,label,text_clean,text_preprocessed
68160,68160,80269,"Petugas PLN Purwakarta memang cepat tanggap, d...",5,2023-08-12 04:42:25,positive,petugas purwakarta cepat tanggap memudahkan ke...,petugas purwakarta cepat tanggap mudah kendala...
104231,104231,134051,maaf pemasangan baru lama prosesnya,1,2023-09-28 17:19:53,negative,maaf pemasangan prosesnya,maaf pasang proses


In [ ]:
# Menghapus kolom 'Unnamed: 0.1' dan 'Unnamed: 0'
df_pln = df_pln.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [ ]:
df_pln.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117468 entries, 0 to 117467
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   content            117468 non-null  object
 1   score              117468 non-null  int64 
 2   at                 117468 non-null  object
 3   label              117468 non-null  object
 4   text_clean         117468 non-null  object
 5   text_preprocessed  117468 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.4+ MB


In [ ]:
print(df_pln['label'].value_counts())

label
positive    96802
negative    17076
neutral      3590
Name: count, dtype: int64


## Step 6: Prepare for Modeling
Menentukan MAX_SEQUENCE_LENGTH:

Menghitung jumlah kata dalam setiap pesan setelah teks dipisahkan oleh spasi.
Menampilkan jumlah kata maksimum dan indeks pesan dengan jumlah kata maksimum untuk referensi.

In [ ]:
# Untuk menentukan MAX_SEQUENCE_LENGTH
length_of_the_messages = df_pln['text_preprocessed'].str.split("\\s+")

print("Max number of words = ", length_of_the_messages.str.len().max())
print("Index = ", length_of_the_messages.str.len().idxmax())

Max number of words =  66
Index =  43855


Menentukan MAX_NB_WORDS:

Membuat salinan dataframe dan membersihkan teks dengan menghapus karakter non-alphanumeric.
Menghitung frekuensi setiap kata dalam teks yang telah dibersihkan.
Menetapkan parameter untuk tokenisasi, yaitu jumlah maksimum kata (MAX_NB_WORDS), panjang maksimum urutan kata (MAX_SEQUENCE_LENGTH), dan dimensi embedding (EMBEDDING_DIM).

In [ ]:
# Untuk menentukan MAX_NB_WORDS
new_df = df_pln.copy()
new_df["mytext_new"] = new_df['text_preprocessed'].str.lower().str.replace('[^\w\s]', '')

# Membagi teks menjadi kata-kata dan menghitung frekuensi setiap kata
new_df = new_df.mytext_new.str.split(expand=True).stack().value_counts().reset_index()
new_df.columns = ['Word', 'Frequency']

In [ ]:
new_df

,Word,Frequency
0,mudah,27422
1,bantu,25523
2,cepat,22842
3,listrik,19874
4,untuk,18855
...,...,...
47041,daerahcikalonglisterik,1
47042,setabilharus,1
47043,nyavtetapi,1
47044,nyatanyacumanv,1



Tokenisasi dan Padding:

Menggunakan Tokenizer untuk mengubah teks menjadi urutan angka berdasarkan frekuensi kata.
Menggunakan pad_sequences untuk memastikan semua urutan teks memiliki panjang yang sama dengan menambahkan padding di depan.



In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.initializers import Constant

In [ ]:
MAX_NB_WORDS = 10000 # Maksimal total jenis kata yang akan digunakan
MAX_SEQUENCE_LENGTH = 50 # Maksimal jumlah kata dalam setiap kalimat
EMBEDDING_DIM = 500 # Dimensi vektor embedding

In [ ]:
# Inisialisasi Tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True)

tokenizer.fit_on_texts(df_pln['text_preprocessed'].values)

# Mendapatkan indeks kata
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 47034 unique tokens.


In [ ]:
# Mengubah teks menjadi urutan angka
X = tokenizer.texts_to_sequences(df_pln['text_preprocessed'].values)

# Menambahkan padding pada urutan angka agar panjangnya konsisten dengan MAX_SEQUENCE_LENGTH
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
# Menampilkan bentuk array data yang telah diproses
print('Shape of data address:', X.shape)

Shape of data address: (117468, 50)


In [ ]:
X

array([[   0,    0,    0, ...,   30, 1198,  252],
       [   0,    0,    0, ...,  213,  162,   17],
       [   0,    0,    0, ...,   36, 1009,   17],
       ...,
       [   0,    0,    0, ...,    0, 8541,  201],
       [   0,    0,    0, ...,    0,   65,  208],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int32)

Label Encoding:

Menggunakan LabelEncoder untuk mengubah label sentimen menjadi format numerik.
Menetapkan kelas yang ada dan mengubah label menjadi format numerik yang sesuai.

In [ ]:
from sklearn.preprocessing import LabelEncoder
Y = df_pln['label']

le = LabelEncoder()
le.classes_ = ['negative', 'neutral', 'positive']
Y_encoded = le.fit_transform(Y)

In [ ]:
print(Y_encoded)

[2 2 2 ... 2 2 2]


In [ ]:
# Menggantikan Y dengan label yang telah dienkode
Y = Y_encoded

Dataset splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=RANDOM_SEED,
)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_test,
    Y_test,
    test_size=0.1,
    random_state=RANDOM_SEED,
)

In [ ]:
print(f'Total Data Train    : {len(X_train)}')
print(f'Total Data Validasi : {len(X_val)}')
print(f'Total Data Test     : {len(X_test)}')

Total Data Train    : 93974
Total Data Validasi : 21144
Total Data Test     : 2350


In [ ]:
Y_test

array([2, 0, 2, ..., 2, 2, 2])

In [ ]:
X.shape[1]

50

## Step 8: LSTM Model Creation and Training


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, GRU, Dense, SpatialDropout1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
from keras.utils import to_categorical

In [ ]:
def create_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM))
    model.add(SpatialDropout1D(0.4))
    model.add(Dropout(0.8))  # Dropout neuron to reduce overfitting
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and train LSTM model
lstm_model = create_lstm_model()
lstm_model.build(input_shape=(None, X.shape[1]))

In [ ]:
X.shape

(117468, 50)

In [ ]:
lstm_history = lstm_model.fit(X_train, Y_train, epochs=20, batch_size=64,
                              validation_data = (X_val, Y_val),
                              callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 29s 16ms/step - accuracy: 0.8817 - loss: 0.3335 - val_accuracy: 0.9087 - val_loss: 0.2528
Epoch 2/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.9142 - loss: 0.2385 - val_accuracy: 0.9105 - val_loss: 0.2439
Epoch 3/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9174 - loss: 0.2289 - val_accuracy: 0.9109 - val_loss: 0.2444
Epoch 4/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9204 - loss: 0.2206 - val_accuracy: 0.9116 - val_loss: 0.2461
Epoch 5/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - accuracy: 0.9222 - loss: 0.2154 - val_accuracy: 0.9117 - val_loss: 0.2438
Epoch 6/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9251 - loss: 0.2074 - val_accuracy: 0.9119 - val_loss: 0.2528
Epoch 7/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9242 - loss: 0.2060 - val_accuracy: 0.9097 - val_loss: 0.2554
Epoch 8/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.9272 -

In [ ]:
lstm_acc = lstm_model.evaluate(X_test, Y_test)
print('LSTM Test set\n  Loss: {:.2f}\n  Accuracy: {:.2f}'.format(lstm_acc[0], lstm_acc[1]))

74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9260 - loss: 0.2078
LSTM Test set
  Loss: 0.22
  Accuracy: 0.92


In [ ]:
import pickle
with open('sentiment_lstm_history.pkl', 'wb') as file_pi:
  pickle.dump(lstm_history.history, file_pi)

In [ ]:
import pickle
with open('sentiment_lstm_history.pkl', "rb") as file_pi:
    lstm_history = pickle.load(file_pi)

In [ ]:
new_text = ['aplikasi pln mobile sangat bagus banget, saya suka']

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = lstm_model.predict(padded)

labels = ['negative', 'neutral', 'positive']

print('Hasil Sentimen Analisis : ',labels[np.argmax(pred)])
print('negative : ',pred[0,0])
print('neutral  : ',pred[0,1])
print('positive : ',pred[0,2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Hasil Sentimen Analisis :  positive
negative :  0.08424507
neutral  :  0.05519754
positive :  0.8605573


In [ ]:
new_text = ['nyesel download pln mobile ini jelek']

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = lstm_model.predict(padded)

labels = ['negative', 'neutral', 'positive']

print('Hasil Sentimen Analisis : ',labels[np.argmax(pred)])
print('negative : ',pred[0,0])
print('neutral  : ',pred[0,1])
print('positive : ',pred[0,2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Hasil Sentimen Analisis :  negative
negative :  0.95944107
neutral  :  0.01644576
positive :  0.024113204


## Step 9: Bi-LSTM Model Creation and Training

In [ ]:
def create_bilstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM))
    model.add(SpatialDropout1D(0.4))
    model.add(Dropout(0.8))  # Dropout neuron to reduce overfitting
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(32)))
    # model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and train LSTM model
bilstm_model = create_bilstm_model()
bilstm_model.build(input_shape=(None, X.shape[1]))

In [ ]:
X.shape

(117468, 50)

In [ ]:
bilstm_history = bilstm_model.fit(X_train, Y_train, epochs=20, batch_size=64,
                              validation_data = (X_val, Y_val),
                              # validation_split=0.15,
                              callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001, restore_best_weights=True)])

Epoch 1/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 45s 28ms/step - accuracy: 0.8827 - loss: 0.3309 - val_accuracy: 0.9089 - val_loss: 0.2553
Epoch 2/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 80s 27ms/step - accuracy: 0.9141 - loss: 0.2408 - val_accuracy: 0.9105 - val_loss: 0.2447
Epoch 3/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.9175 - loss: 0.2286 - val_accuracy: 0.9117 - val_loss: 0.2469
Epoch 4/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9213 - loss: 0.2194 - val_accuracy: 0.9120 - val_loss: 0.2408
Epoch 5/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.9231 - loss: 0.2100 - val_accuracy: 0.9098 - val_loss: 0.2511
Epoch 6/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.9262 - loss: 0.2024 - val_accuracy: 0.9134 - val_loss: 0.2481
Epoch 7/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 42s 28ms/step - accuracy: 0.9253 - loss: 0.2022 - val_accuracy: 0.9114 - val_loss: 0.2501


In [ ]:
import pickle
with open('sentiment_gru_history.pkl', 'wb') as file_pi:
  pickle.dump(bilstm_history.history, file_pi)

In [ ]:
import pickle
with open('sentiment_gru_history.pkl', "rb") as file_pi:
    bilstm_history = pickle.load(file_pi)

In [ ]:
bilstm_acc = bilstm_model.evaluate(X_test,Y_test)
print('Bi-LSTM Test set\n  Loss: {:.2f}\n  Accuracy: {:.2f}'.format(bilstm_acc[0],bilstm_acc[1]))

74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9172 - loss: 0.2158
Bi-LSTM Test set
  Loss: 0.22
  Accuracy: 0.92


In [ ]:
new_text = ['aplikasi pln mobile sangat bagus sekali, saya suka']

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = bilstm_model.predict(padded)

labels = ['negative', 'neutral', 'positive']

print('Hasil Sentimen Analisis : ',labels[np.argmax(pred)])
print('negative : ',pred[0,0])
print('neutral  : ',pred[0,1])
print('positive : ',pred[0,2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
Hasil Sentimen Analisis :  positive
negative :  0.15524563
neutral  :  0.08469922
positive :  0.7600552


In [ ]:
new_text = ['nyesel download pln mobile ini jelek']

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = bilstm_model.predict(padded)

labels = ['negative', 'neutral', 'positive']

print('Hasil Sentimen Analisis : ',labels[np.argmax(pred)])
print('negative : ',pred[0,0])
print('neutral  : ',pred[0,1])
print('positive : ',pred[0,2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Hasil Sentimen Analisis :  negative
negative :  0.9695165
neutral  :  0.014858492
positive :  0.015624991


## Step 9: GRU Model Creation and Training

In [ ]:
def create_gru_model():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM))
    model.add(SpatialDropout1D(0.4))
    model.add(Dropout(0.8))  # Dropout neuron to reduce overfitting
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(Bidirectional(GRU(64)))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and train GRU model
gru_model = create_gru_model()
gru_model.build(input_shape=(None, X.shape[1]))

In [ ]:
gru_history = gru_model.fit(X_train, Y_train, epochs=20, batch_size=64,
                              validation_data = (X_val, Y_val),
                              # validation_split=0.15,
                              callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 34s 21ms/step - accuracy: 0.8824 - loss: 0.3266 - val_accuracy: 0.9112 - val_loss: 0.2460
Epoch 2/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9140 - loss: 0.2399 - val_accuracy: 0.9118 - val_loss: 0.2441
Epoch 3/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9178 - loss: 0.2283 - val_accuracy: 0.9123 - val_loss: 0.2462
Epoch 4/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.9213 - loss: 0.2185 - val_accuracy: 0.9126 - val_loss: 0.2465
Epoch 5/20
1469/1469 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9237 - loss: 0.2116 - val_accuracy: 0.9113 - val_loss: 0.2455


In [ ]:
import pickle
with open('sentiment_bilstm_history.pkl', 'wb') as file_pi:
  pickle.dump(gru_history.history, file_pi)

In [ ]:
import pickle
with open('sentiment_bilstm_history.pkl', "rb") as file_pi:
    gru_history = pickle.load(file_pi)

In [ ]:
gru_acc = gru_model.evaluate(X_test,Y_test)
print('GRU Test set\n  Loss: {:.2f}\n  Accuracy: {:.2f}'.format(gru_acc[0],gru_acc[1]))

74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9183 - loss: 0.2134
GRU Test set
  Loss: 0.22
  Accuracy: 0.92


In [ ]:
new_text = ['aplikasi pln mobile sangat bagus sekali, saya suka']

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = gru_model.predict(padded)

labels = ['negative', 'neutral', 'positive']

print('Hasil Sentimen Analisis : ',labels[np.argmax(pred)])
print('negative : ',pred[0,0])
print('neutral  : ',pred[0,1])
print('positive : ',pred[0,2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Hasil Sentimen Analisis :  positive
negative :  0.15030973
neutral  :  0.10793445
positive :  0.74175584


In [ ]:
new_text = ['nyesel download pln mobile ini jelek']

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = gru_model.predict(padded)

labels = ['negative', 'neutral', 'positive']

print('Hasil Sentimen Analisis : ',labels[np.argmax(pred)])
print('negative : ',pred[0,0])
print('neutral  : ',pred[0,1])
print('positive : ',pred[0,2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Hasil Sentimen Analisis :  negative
negative :  0.95225734
neutral  :  0.023086984
positive :  0.024655795


## Step 10: Model Evaluation (LSTM, Bi-LSTM, and GRU)
Evaluate both the LSTM and GRU models on the test set.

In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate

# Evaluate LSTM model
lstm_acc = lstm_model.evaluate(X_test, Y_test)
# Evaluate Bi-LSTM model
bilstm_acc = bilstm_model.evaluate(X_test, Y_test)
# Evaluate GRU model
gru_acc = gru_model.evaluate(X_test, Y_test)

# Create a table with the results
results = [
    ["Model", "Loss", "Accuracy"],
    ["LSTM", f"{lstm_acc[0]:.3f}", f"{lstm_acc[1] * 100:.3f}%"],
    ["Bi-LSTM", f"{bilstm_acc[0]:.3f}", f"{bilstm_acc[1] * 100:.3f}%"],
    ["GRU", f"{gru_acc[0]:.3f}", f"{gru_acc[1] * 100:.3f}%"]
]

# Print the table
print(tabulate(results, headers="firstrow", tablefmt="grid"))

74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9260 - loss: 0.2078
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9172 - loss: 0.2158
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9183 - loss: 0.2134
+---------+--------+------------+
| Model   |   Loss | Accuracy   |
+=========+========+============+
| LSTM    |  0.221 | 92.298%    |
+---------+--------+------------+
| Bi-LSTM |  0.222 | 91.574%    |
+---------+--------+------------+
| GRU     |  0.222 | 91.574%    |
+---------+--------+------------+


## Step 12: Custom Test for New Text Input


In [ ]:
input_text = str(input("Input Ulasan: "))
new_text = [input_text]

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)

labels = ['negative', 'neutral', 'positive']

# Predict using LSTM
pred_lstm = lstm_model.predict(padded)
# Predict using BiLSTM
pred_bilstm = bilstm_model.predict(padded)
# Predict using GRU
pred_gru = gru_model.predict(padded)

# Prepare data for table
results = [
    ["Model", "Hasil Sentimen Analisis", "Negatif", "Netral", "Positif"],
    ["LSTM", labels[np.argmax(pred_lstm)], pred_lstm[0,0], pred_lstm[0,1], pred_lstm[0,2]],
    ["Bi-LSTM", labels[np.argmax(pred_bilstm)], pred_bilstm[0,0], pred_bilstm[0,1], pred_bilstm[0,2]],
    ["GRU", labels[np.argmax(pred_gru)], pred_gru[0,0], pred_gru[0,1], pred_gru[0,2]]
]

# Print the table with a different format
print(tabulate(results, headers="firstrow", tablefmt="simple"))

Input Ulasan: pelayanan dalam aplikasi sangat bagus dan rapih, mantap
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Model    Hasil Sentimen Analisis        Negatif       Netral    Positif
-------  -------------------------  -----------  -----------  ---------
LSTM     positive                   0.000550808  0.000390295   0.999059
Bi-LSTM  positive                   0.000722208  0.000603179   0.998675
GRU      positive                   0.00102138   0.000932687   0.998046


In [ ]:
input_text = str(input("Input Ulasan: "))
new_text = [input_text]

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)

labels = ['negative', 'neutral', 'positive']

# Predict using LSTM
pred_lstm = lstm_model.predict(padded)
# Predict using BiLSTM
pred_bilstm = bilstm_model.predict(padded)
# Predict using GRU
pred_gru = gru_model.predict(padded)

# Prepare data for table
results = [
    ["Model", "Hasil Sentimen Analisis", "Negatif", "Netral", "Positif"],
    ["LSTM", labels[np.argmax(pred_lstm)], pred_lstm[0,0], pred_lstm[0,1], pred_lstm[0,2]],
    ["Bi-LSTM", labels[np.argmax(pred_bilstm)], pred_bilstm[0,0], pred_bilstm[0,1], pred_bilstm[0,2]],
    ["GRU", labels[np.argmax(pred_gru)], pred_gru[0,0], pred_gru[0,1], pred_gru[0,2]]
]

# Print the table with a different format
print(tabulate(results, headers="firstrow", tablefmt="simple"))

Input Ulasan: loading lama dan susah dibuka aplikasi pln mobile ini
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Model    Hasil Sentimen Analisis      Negatif     Netral    Positif
-------  -------------------------  ---------  ---------  ---------
LSTM     negative                    0.902661  0.0523296  0.0450089
Bi-LSTM  negative                    0.863434  0.071416   0.0651496
GRU      negative                    0.91307   0.0520445  0.0348858


In [ ]:
input_text = str(input("Input Ulasan: "))
new_text = [input_text]

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)

labels = ['negative', 'neutral', 'positive']

# Predict using LSTM
pred_lstm = lstm_model.predict(padded)
# Predict using BiLSTM
pred_bilstm = bilstm_model.predict(padded)
# Predict using GRU
pred_gru = gru_model.predict(padded)

# Prepare data for table
results = [
    ["Model", "Hasil Sentimen Analisis", "Negatif", "Netral", "Positif"],
    ["LSTM", labels[np.argmax(pred_lstm)], pred_lstm[0,0], pred_lstm[0,1], pred_lstm[0,2]],
    ["Bi-LSTM", labels[np.argmax(pred_bilstm)], pred_bilstm[0,0], pred_bilstm[0,1], pred_bilstm[0,2]],
    ["GRU", labels[np.argmax(pred_gru)], pred_gru[0,0], pred_gru[0,1], pred_gru[0,2]]
]

# Print the table with a different format
print(tabulate(results, headers="firstrow", tablefmt="simple"))

Input Ulasan: Aplikasi ini telah diperbarui baru-baru ini
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Model    Hasil Sentimen Analisis      Negatif     Netral    Positif
-------  -------------------------  ---------  ---------  ---------
LSTM     positive                    0.342303  0.0267542   0.630943
Bi-LSTM  negative                    0.62542   0.0625161   0.312064
GRU      negative                    0.525498  0.0346613   0.439841
